<h4>-----------------------------------------------------------------------------<br>Copyright (c) 2023, Lucid Vision Labs, Inc.</h4>
<h5> THE  SOFTWARE  IS  PROVIDED  "AS IS",  WITHOUT  WARRANTY  OF  ANY  KIND,<br>EXPRESS  OR  IMPLIED,  INCLUDING  BUT  NOT  LIMITED  TO  THE  WARRANTIES<br>OF  MERCHANTABILITY,  FITNESS  FOR  A  PARTICULAR  PURPOSE  AND<br>NONINFRINGEMENT.  IN  NO  EVENT  SHALL  THE  AUTHORS  OR  COPYRIGHT  HOLDERS<br>BE  LIABLE  FOR  ANY  CLAIM,  DAMAGES  OR  OTHER  LIABILITY,  WHETHER  IN  AN<br>ACTION  OF  CONTRACT,  TORT  OR  OTHERWISE,  ARISING  FROM,  OUT  OF  OR  IN<br>CONNECTION  WITH  THE  SOFTWARE  OR  THE  USE  OR  OTHER  DEALINGS  IN <br> THE  SOFTWARE.<br>-----------------------------------------------------------------------------</h5>

# Object Detection with YOLO

This examples demonstrates acquiring a video stream using the Arena SDK, using OpenCV to convert  captured frames into an nparray, and using <a href="https://github.com/ultralytics/ultralytics">Ultralytics YOLOv8</a>, to identify objects within the image. This example includes device enumeration, image detection, and clean up.

# Version History

<table style="float:left">
<tr>
    <th>Version</th>
    <th>Description</th>
    <th>Date</th>
</tr>
<tr>
    <td>1.0</td>
    <td>Initial Release</td>
    <td>November 6, 2023</td>
</tr>
</table>

# Cameras

- Phoenix
- Triton
- Atlas
- Atlas10

# Dependencies

This application requires the following dependencies:
- opencv-python 4.7.0.72
- numpy 1.21.6
- ultralytics 8.0.118

In [ ]:
!pip install opencv-python==4.7.0.72
!pip install numpy==1.21.6
!pip install ultralytics

# Testing

This application has been tested with Python v3.7.11

#### Set **use_camera_image = True** to use a camera image

#### Set **use_camera_image = False** to use a predefined image (sample_image.png in this directory):
<img src="sample_image.png" />

In [ ]:
use_camera_image = True

In [ ]:
import cv2
import numpy as np

In [ ]:
from ultralytics import YOLO

In [ ]:
from arena_api.buffer import BufferFactory
from arena_api.enums import PixelFormat
from arena_api.system import system

In [ ]:
window_width = 800
window_height = 600

do_annotate = True

In [ ]:
yolo_model = YOLO("yolov8n.pt")

In [ ]:
def annotate_image(image):
    global yolo_model
    
    # vebose=False to supress console output for every object detected by YOLO
    results = yolo_model(image, verbose=False)

    annotated_frame = results[0].plot()
    
    return annotated_frame

In [ ]:
def get_image_buffers(device, is_color_camera=False):
    global do_annotate
    
    key = -1
    cv2.namedWindow("Image", cv2.WINDOW_NORMAL)
    display_image = None
    
    device.start_stream()
    
    while True:
        image_buffer = device.get_buffer()
        
        # convert the image into a NumPy array
        nparray = np.ctypeslib.as_array(image_buffer.pdata,shape=(image_buffer.height, image_buffer.width, int(image_buffer.bits_per_pixel / 8))).reshape(image_buffer.height, image_buffer.width, int(image_buffer.bits_per_pixel / 8))

        if is_color_camera == True:
            display_img = cv2.cvtColor(nparray, cv2.COLOR_BayerBG2BGR)
        else:
            display_img = cv2.cvtColor(nparray, cv2.COLOR_GRAY2BGR)
            
            
        if do_annotate == True:
            annotated_image = annotate_image(display_img)
            
            cv2.resizeWindow("Image", window_width, window_height)
            cv2.imshow("Image", annotated_image)
        else:
            cv2.resizeWindow("Image", window_width, window_height)
            cv2.imshow("Image", display_img)
            
        
        # press q in the OpenCV window to quit strema
        # press y in the OpenCV window to turn off YOLO image evaluation
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break
        elif key == ord("y"):
            do_annotate = not do_annotate

        device.requeue_buffer(image_buffer)

    cv2.destroyAllWindows()
    device.stop_stream()

In [ ]:
def read_sample_image():
    
    key = -1
    cv2.namedWindow("Image", cv2.WINDOW_NORMAL)
    
    display_img = cv2.imread("sample_image.png", cv2.IMREAD_COLOR)
    
    processed_image = annotate_image(display_img)
    
    while True:
        cv2.resizeWindow("Image", window_width, window_height)
        cv2.imshow("Image", processed_image)

        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break
            
    cv2.destroyAllWindows()

### The following function detects the number of cameras. If only 1 camera is detected, the function will attempt to connect to the camera. If more than 1 camera is detected, the function will wait  for the user to select a device.

In [ ]:
if use_camera_image == True:

    device_infos = None
    selected_index = None

    while selected_index is None:
        device_infos = system.device_infos
        num_devices = len(device_infos)

        if num_devices == 0:
            print("No camera connected\nPress enter to search again")
            input()
            continue

        elif num_devices == 1:
            selected_index = 0
            continue

        else:
            print("Devices found:")
            for i in range(len(device_infos)):
                print(f"\t{i}. {device_infos[i]['model']} SN: {device_infos[i]['serial']}")

            while True:
                line = input("Selection: ")
                try:
                    selected_index = int(line)
                    if 0 <= selected_index < len(device_infos):
                        break
                    else:
                        print(f"Please enter a valid number between 0 and {len(device_infos)-1}\n")
                except Exception as e:
                    print("\nPlease enter a valid number\n")

    selected_model = device_infos[selected_index]['model']
    print(f"\nCreate device: {selected_model}...")
    device = system.create_device(device_infos=device_infos[selected_index])[0]

In [ ]:
if use_camera_image == True:
    # create device
    device.tl_stream_nodemap.get_node('StreamBufferHandlingMode').value = 'NewestOnly'
    device.tl_stream_nodemap.get_node('StreamPacketResendEnable').value = True
    device.tl_stream_nodemap.get_node('StreamAutoNegotiatePacketSize').value = True

    isp_bayer_pattern = device.nodemap.get_node('IspBayerPattern').value
    is_color_camera = False

    if isp_bayer_pattern != 'NONE':
        is_color_camera = True

    if is_color_camera == True:
        device.nodemap.get_node('PixelFormat').value = "BayerRG8"
    else:
        device.nodemap.get_node('PixelFormat').value = "Mono8"

In [ ]:
if use_camera_image == True:
    # grab images -------------------------------------------------------------
    get_image_buffers(device, is_color_camera)
else:
    read_sample_image()

In [ ]:
if use_camera_image == True:
    system.destroy_device()
    print('Destroyed all created devices')